ライブラリのインポート
データの読み込み、別れている場合は前処理の前に結合
データの確認
欠損値、異常値の処理
データ型変換・ダミー化(回帰分析の場合)
データの可視化
モデルの学習方法の選択

In [269]:
#ライブラリのインポートimport
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [270]:
#データの読み込みpd.read_
df_train = pd.read_csv("train.tsv", sep="\t")
df_test = pd.read_csv("test.tsv", sep="\t")

In [271]:
#前処理の前に結合pd.concat
df_train["ds_test"]=0
df_test["ds_test"]=1
df_all=pd.concat([df_train, df_test], axis=0)
df_all.head()

,id,survived,pclass,sex,age,sibsp,parch,fare,embarked,ds_test
0,3,1.0,1,female,35.0,1,0,53.1000,S,0
1,4,0.0,3,male,35.0,0,0,8.0500,S,0
2,7,0.0,3,male,2.0,3,1,21.0750,S,0
3,9,1.0,2,female,14.0,1,0,30.0708,C,0
4,11,1.0,1,female,58.0,0,0,26.5500,S,0


In [272]:
#データの確認1.describe
"""
countをidと比較して欠損値を確認
minとmax を確認し、値が常識的な範囲内かチェックします。
std（標準偏差）が極端に大きい場合、データに異常値（外れ値）が含まれている可能性があります。
mean(平均値)と50%(中央値)を比較データ分布の偏りを確認
四分位範囲（IQR）を使った異常値検出
25%と75%（四分位数）を使ってデータ範囲を確認します。
場合によってはIQR（第3四分位 - 第1四分位）を基に、範囲外の値を異常値と見なせます。
"""
df_all.describe()


,id,survived,pclass,age,sibsp,parch,fare,ds_test
count,891.000000,445.000000,891.000000,714.000000,891.000000,891.000000,891.000000,891.000000
mean,445.000000,0.402247,2.308642,29.699118,0.523008,0.381594,32.204208,0.500561
std,257.353842,0.490903,0.836071,14.526497,1.102743,0.806057,49.693429,0.500281
min,0.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000,0.000000
25%,222.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400,0.000000
50%,445.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200,1.000000
75%,667.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000,1.000000
max,890.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200,1.000000


In [273]:
#データの確認2.info
"""
Index: データの行数 と Nou-Null Count:非欠損値の数
を比較して欠損値の数を確認できる。あれば欠損値を処理
dtypes:をみてobject型があればダミー化を検討
"""
df_all.info()


<class 'pandas.core.frame.DataFrame'>
Index: 891 entries, 0 to 445
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        891 non-null    int64  
 1   survived  445 non-null    float64
 2   pclass    891 non-null    int64  
 3   sex       891 non-null    object 
 4   age       714 non-null    float64
 5   sibsp     891 non-null    int64  
 6   parch     891 non-null    int64  
 7   fare      891 non-null    float64
 8   embarked  889 non-null    object 
 9   ds_test   891 non-null    int64  
dtypes: float64(3), int64(5), object(2)
memory usage: 76.6+ KB


In [274]:
#欠損値処理の処理
#fillnaを実行しただけだと元のデータは変わらないので代入する
df_all["age"] = df_all["age"].fillna(df_all["age"].mean())
df_all["embarked"] = df_all["embarked"].fillna(df_all["embarked"].mode()[0])
#mode()は最頻値が複数あるときのためにシリーズ型(可変のインデックスを持つリスト)を返すので[0]で先頭を選択
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Index: 891 entries, 0 to 445
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        891 non-null    int64  
 1   survived  445 non-null    float64
 2   pclass    891 non-null    int64  
 3   sex       891 non-null    object 
 4   age       891 non-null    float64
 5   sibsp     891 non-null    int64  
 6   parch     891 non-null    int64  
 7   fare      891 non-null    float64
 8   embarked  891 non-null    object 
 9   ds_test   891 non-null    int64  
dtypes: float64(3), int64(5), object(2)
memory usage: 76.6+ KB


In [275]:
#ダミー化
# pd.get_dummies(data=df_all) これでもOK
df_all["embarked"], uniques_em = pd.factorize(df_all["embarked"])
df_all["sex"], uniques_s = pd.factorize(df_all["sex"])
print (df_all.head(), "\n",
       uniques_em, uniques_s)

   id  survived  pclass  sex   age  sibsp  parch     fare  embarked  ds_test
0   3       1.0       1    0  35.0      1      0  53.1000         0        0
1   4       0.0       3    1  35.0      0      0   8.0500         0        0
2   7       0.0       3    1   2.0      3      1  21.0750         0        0
3   9       1.0       2    0  14.0      1      0  30.0708         1        0
4  11       1.0       1    0  58.0      0      0  26.5500         0        0 
 Index(['S', 'C', 'Q'], dtype='object') Index(['female', 'male'], dtype='object')


In [289]:
#データの可視化1.groupby
cate_colums =  ["pclass","sex","sibsp","parch", "embarked"]
num_colums = ["age", "fare"]
dfs=[]
for col in cate_colums:
    tmp = df_all[["survived", col]].groupby(col).mean()
    dfs.append(tmp)


In [ ]:
#データの可視化2
#回帰問題なら、カテゴリ変数はカテゴリごとに箱ひげ図、連続変数は、散布図
#分類問題なら目的変数のクラスごとに、カテゴリ変数は積み上げ棒グラフ、連続変数はKDE


次元削減、クラスタリング、パターンの発見→教師なし学習
予測(回帰、分類)→教師あり学習
